In [ ]:
%load_ext autoreload
%autoreload 2
from awesome.run.awesome_config import AwesomeConfig
from awesome.run.awesome_runner import AwesomeRunner
from awesome.util.reflection import class_name
from awesome.analytics.result_model import ResultModel
from awesome.util.path_tools import get_project_root_path, get_package_root_path
import os
from tqdm.auto import tqdm
import scipy
import torch
import numpy as np
import cv2 as cv
import math
from awesome.run.functions import plot_as_image, plot_mask
import matplotlib.pyplot as plt
import re
from awesome.util.format import latex_postprocessor

os.chdir(get_project_root_path()) # Beeing in the root directory of the project is important for the relative paths to work consistently

In [ ]:
path = "./runs/convexity/07_CNNET_benchmark+featxy+convex_23_10_01_16_57_47"

result_model = ResultModel.from_path(path)
result_model

In [ ]:
runner = result_model.get_runner(-1)
model = runner.agent._get_model()
dataloader = runner.dataloader

In [ ]:
import numpy as np
import cv2 as cv
import math
from awesome.run.functions import plot_as_image
from tqdm.auto import tqdm
import scipy
import logging

index = 1

mask = result_model[index]

fig = plot_as_image(mask)
display(fig)
plt.close(fig)

def extract_contour_points(mask: np.ndarray, min_size: int = 0):
    img_mask = (mask * 255).astype(np.uint8)
    contours, _ = cv.findContours(img_mask, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_NONE)
    if min_size != 0:
        filtered_contours = []
        for c in contours:
            area = cv.contourArea(c)
            if area >= min_size:
                filtered_contours.append(c)
        contours = filtered_contours
    
    
    indev_length = np.array([len(x) for x in contours])
    ends = np.cumsum(indev_length)
    combined = np.concatenate(contours, axis=0)[:, 0, :]

    is_filled = np.zeros((len(ends)), dtype=np.bool_)
    for i, c in enumerate(contours):
        m = np.zeros(img_mask.shape)
        cv.fillConvexPoly(m, c, color=255, lineType=cv.LINE_8)
        # Contour points are part of the mask 1 mask so ignore them to determine wether it is a filled shape
        # Only covers holes in masks, if hole itself has a contour, it will also return filled
        m[c[:, 0, 1], c[:, 0, 0]] = False
        is_filled[i] = mask[m == 255].any()
        # fig = plot_as_image(m)
        # display(fig)
        # plt.close(fig)
    return combined, ends, is_filled 

def find_cut_points(contour_points, ends):
    cut = []
    tangent = []

    for i, end in enumerate(ends):
        start = 0
        if i > 0:
            start = ends[i-1]
        cut_pts = contour_points[start: end][::-1] # Reverse to me anti clockwise
        tangent_pts = np.roll(cut_pts, -1, axis=0)
        cut.append(cut_pts)
        tangent.append(tangent_pts)
        #display(np.concatenate([cut_pts, tangent_pts], axis=1))
    cut_points = np.concatenate(cut)
    tangent_points = np.concatenate(tangent)
    assert len(contour_points) == len(cut_points)
    assert len(contour_points) == len(tangent_points)
    return cut_points, tangent_points


def process_result_model(result_model, folder, progress_bar = None):
    it = range(len(result_model))
    if progress_bar is None:
        progress_bar = tqdm(total=len(result_model), desc="Process images")
    else:
        progress_bar.reset(total=len(result_model))

    os.makedirs(folder, exist_ok=True)

    for index in it:
        try:
            mask = result_model[index]
            contour_points, ends, is_filled = extract_contour_points(mask, 0)
            cut_points, tangent_points = find_cut_points(contour_points, ends)

            cut_path = os.path.join(folder, f"cut_points_{index}.mat")
            tangent_path = os.path.join(folder, f"tangent_points_{index}.mat")
            end_path = os.path.join(folder, f"end_index_points_{index}.mat")
            is_filled_path = os.path.join(folder, f"is_filled_{index}.mat")

            scipy.io.savemat(cut_path, dict(cut_points=cut_points), appendmat=False)
            scipy.io.savemat(tangent_path, dict(tangent_points=tangent_points), appendmat=False)
            scipy.io.savemat(end_path, dict(end_index_points=ends), appendmat=False)
            scipy.io.savemat(is_filled_path, dict(is_filled=is_filled), appendmat=False)
        except Exception as err:
            logging.warning(f"Failed to process index {index} of Result model {result_model.name}")
        progress_bar.update()
    return progress_bar

contour_points, ends, is_filled = extract_contour_points(mask, 10)


In [ ]:


find_cut_points(contour_points, ends, is_filled)

In [ ]:
import matplotlib.pyplot as plt
msks = np.zeros((*mask.shape[:2], len(ends)))
for i, end in enumerate(ends):
    start = 0
    if i > 0:
        start = ends[i-1]
    pts = contour_points[start: end]
    msks[pts[:, 0, 1], pts[:, 0, 0], i] = True 

    #fig = plot_as_image(msks[:,:,i])
    #display(fig)
    #plt.close(fig)

In [ ]:
from awesome.run.functions import plot_mask

plt.close("all")
fig = plot_mask(None, msks[...], size=10, axes_description=True, labels=[str(x) for x in range(msks.shape[2])], legend=True)
display(fig)
plt.close(fig)

In [ ]:
msks[..., 2:].any()


In [ ]:
from awesome.run.functions import plot_as_image

plot_as_image(mask)

In [ ]:
rm_paths = [
    "./runs/convexity/#09_FCNET_benchmark+featxy+convex_23_10_01_11_44_56",
    "./runs/convexity/#14_FCNET_benchmark+featxy+convex+seed47_23_10_01_13_20_04",
    "./runs/convexity/06_CNNET_benchmark+featxy+convex+seed131_23_10_01_16_57_11",
    "./runs/convexity/07_CNNET_benchmark+featxy+convex_23_10_01_16_57_47",
    "./runs/convexity/11_FCNET_benchmark+featxy+convex+seed131_23_10_01_22_55_21",
    "./runs/convexity/15_CNNET_benchmark+featxy+convex+seed47_23_10_01_23_22_53",
]

dest_folder = [
    "FCNET_featxy_seed42",
    "FCNET_featxy_seed47",
    "CNNET_featxy_seed131",
    "CNNET_featxy_seed42",
    "FCNET_featxy_seed131",
    "CNNET_featxy_seed47",
]
p = "output/cut_points"
for path, d in zip(rm_paths, dest_folder):
    it = None
    rm = ResultModel.from_path(path)
    it = process_result_model(rm, os.path.join(p, d), it)
    it.close()

In [ ]:
mats_path = "./output/matlab_results/results/mat"

def sort(x):
    file, ext = os.path.splitext(x)
    return int(file)

mat_files = [os.path.join(mats_path, f) for f in sorted(os.listdir(mats_path), key=sort)]



In [ ]:
import cv2 as cv
from awesome.util.temporary_property import TemporaryProperty
import pandas as pd
from awesome.measures.miou import MIOU
from awesome.measures.pixel_accuracy import PixelAccuracy

index = 2
df = pd.DataFrame(columns=[
    "IoU", 
    "Acc."
], index=range(len(dataloader)))

iou = MIOU()
acc = PixelAccuracy()

for index in range(len(dataloader)):
    loaded_mat = scipy.io.loadmat(mat_files[index])
    coords = loaded_mat["mConvexGeo_ConvexityShapedModel"].T

    with TemporaryProperty(dataloader, mode="sample", return_prior=False):
        sample = dataloader[index]
        img = sample["image"]
        gt = 1 - sample["gt"][..., None]
        coords = np.round(coords).astype(int)
        mask = np.zeros(img.shape[1:])[..., None]

        cv.fillConvexPoly(mask, coords, color=1, lineType=cv.LINE_8)

        #mask[coords[:, 1], coords[:, 0]] = True

        df.at[index, "IoU"] = iou(mask, gt).item()
        df.at[index, "Acc."] = acc(mask, gt).item()


        seg_msk = result_model[index]
        stacked_msk = np.concatenate([gt, mask], axis=2)
        p = os.path.join(mats_path, f"../output_mask/img_{index}.png")
        plot_mask(img, stacked_msk, path=p, labels=[ 
            "Ground Truth",
            f"Convex-Seg IoU: {df.at[index, 'IoU']:.3f} Acc.: {df.at[index, 'Acc.']:.3f}"
        ], save=True, close=True)

In [ ]:
plot_as_image(gt, display=True)

In [ ]:
from awesome.run.functions import plot_mask

In [ ]:
df.mean()

In [ ]:
MIOU()(mask, gt)

In [ ]:
from awesome.run.functions import saveable, plot_as_image

In [ ]:
coords = np.round(coords).astype(int)

In [ ]:
sample[1]